In [1]:
import dagshub
import os
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, root_mean_squared_error
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from dagshub import get_repo_bucket_client


In [2]:
dagshub.init(url="https://dagshub.com/colome8/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Accessing as colome8

Initialized MLflow to track repo "colome8/nyc-taxi-time-prediction"

Repository colome8/nyc-taxi-time-prediction initialized!

https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow


<Experiment: artifact_location='mlflow-artifacts:/52f4281e23894b08be828e86a3d02bc9', creation_time=1726632055317, experiment_id='0', last_update_time=1726632055317, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    return df

In [4]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [5]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
categorical = ['PU_DO']
numerical = ['trip_distance']
target = 'duration'


In [7]:
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [10]:
mlflow.sklearn.autolog()

2024/11/12 21:17:22 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.


### Gradient Boosting

In [11]:
import os
import mlflow
import dagshub
from mlflow.tracking import MlflowClient

# Configurar DagsHub y MLflow
dagshub.init(repo_owner="colome8", repo_name="nyc-taxi-time-prediction", mlflow=True)

# Obtener la URI de seguimiento de MLflow
mlflow_tracking_uri = mlflow.get_tracking_uri()
print(f"MLflow Tracking URI: {mlflow_tracking_uri}")

# Configurar MLflow para usar la URI de DagsHub
mlflow.set_tracking_uri(mlflow_tracking_uri)

# Nombre del experimento
experiment_name = "nyc-taxi-experiment"

# Crear o obtener el experimento
client = MlflowClient()
try:
    experiment = client.create_experiment(experiment_name)
except mlflow.exceptions.MlflowException:
    experiment = client.get_experiment_by_name(experiment_name)

if experiment:
    print(f"Experiment ID: {experiment.experiment_id}")
    print(f"Artifact Location: {experiment.artifact_location}")
    
    # Configurar el experimento activo
    mlflow.set_experiment(experiment_name)
else:
    print("Failed to create or retrieve the experiment.")

Initialized MLflow to track repo "colome8/nyc-taxi-time-prediction"

Repository colome8/nyc-taxi-time-prediction initialized!

MLflow Tracking URI: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow
Experiment ID: 0
Artifact Location: mlflow-artifacts:/52f4281e23894b08be828e86a3d02bc9


In [12]:
# Función para ejecutar experimentos
def objective(params, model_class, X_train, y_train, X_val, y_val):
    with mlflow.start_run(nested=True):
        model = model_class(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_params(params)
        mlflow.log_metric("rmse", rmse)
        return {'loss': rmse, 'status': STATUS_OK}

In [13]:
# Espacios de búsqueda para hiperparámetros
gb_space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 500, 50)),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'max_depth': scope.int(hp.quniform('max_depth', 3, 10, 1))
}

rf_space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 500, 50)),
    'max_depth': scope.int(hp.quniform('max_depth', 3, 20, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1))
}


In [14]:
# Experimentos
with mlflow.start_run(run_name="Parent Experiment"):
    # Gradient Boost
    gb_trials = Trials()
    best_gb = fmin(
        fn=lambda params: objective(params, GradientBoostingRegressor, X_train, y_train, X_val, y_val),
        space=gb_space,
        algo=tpe.suggest,
        max_evals=20,
        trials=gb_trials
    )
    
    # Random Forest
    rf_trials = Trials()
    best_rf = fmin(
        fn=lambda params: objective(params, RandomForestRegressor, X_train, y_train, X_val, y_val),
        space=rf_space,
        algo=tpe.suggest,
        max_evals=20,
        trials=rf_trials
    )

    # Seleccionar el mejor modelo
    gb_rmse = min(trial['result']['loss'] for trial in gb_trials.trials)
    rf_rmse = min(trial['result']['loss'] for trial in rf_trials.trials)

    if gb_rmse < rf_rmse:
        best_model = GradientBoostingRegressor(**best_gb)
        best_model_name = "GradientBoost"
    else:
        best_model = RandomForestRegressor(**best_rf)
        best_model_name = "RandomForest"

    # Entrenar el mejor modelo con todos los datos
    best_model.fit(X_train, y_train)

    # Registrar el mejor modelo
    mlflow.sklearn.log_model(best_model, "nyc-taxi-model")
    
    # Asignar el alias 'challenger'
    client = mlflow.tracking.MlflowClient()
    model_version = client.create_model_version("nyc-taxi-model", f"runs:/{mlflow.active_run().info.run_id}/nyc-taxi-model")
    client.set_registered_model_alias("nyc-taxi-model", "challenger", model_version.version)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

2024/11/12 21:17:28 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/11/12 21:19:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run sedate-shark-353 at: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/6398fd642654432792c53e220bd6db3e.

2024/11/12 21:19:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow/#/experiments/0.



  5%|▌         | 1/20 [02:03<39:13, 123.89s/trial, best loss: 5.240192172926639]

2024/11/12 21:19:31 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/11/12 21:20:47 INFO mlflow.tracking._tracking_service.client: 🏃 View run dapper-swan-601 at: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/119ffdfd31c140fbaaead3cdf1d64ddc.

2024/11/12 21:20:47 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow/#/experiments/0.



 10%|█         | 2/20 [03:20<28:44, 95.83s/trial, best loss: 5.240192172926639] 

...

 95%|█████████▌| 19/20 [15:13:09<16:44, 1004.68s/trial, best loss: 5.39879397171728]

2024/11/13 12:53:32 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/11/13 12:54:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run angry-roo-115 at: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/46d3cecc573d40909f5e33a372229f09.

2024/11/13 12:54:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow/#/experiments/0.



100%|██████████| 20/20 [15:14:07<00:00, 2742.38s/trial, best loss: 5.39879397171728]


2024/11/13 12:54:30 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/11/13 12:54:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run Parent Experiment at: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/8a75041f3c394537ade24a79f0fbcb90.
2024/11/13 12:54:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/colome8/nyc-taxi-time-prediction.mlflow/#/experiments/0.


InvalidParameterError: The 'max_depth' parameter of GradientBoostingRegressor must be an int in the range [1, inf) or None. Got np.float64(9.0) instead.

In [ ]:
import mlflow
from mlflow.exceptions import MlflowException
from sklearn.metrics import root_mean_squared_error

# Función para cargar un modelo de forma segura
def safe_load_model(model_name, stage):
    try:
        return mlflow.sklearn.load_model(f"models:/{model_name}@{stage}")
    except MlflowException as e:
        print(f"Error loading {stage} model: {e}")
        return None

# Evaluar modelos en el conjunto de prueba
champion_model = safe_load_model("nyc-taxi-model", "champion")
challenger_model = safe_load_model("nyc-taxi-model", "challenger")

if champion_model and challenger_model:
    champion_rmse = root_mean_squared_error(y_test, champion_model.predict(X_test))
    challenger_rmse = root_mean_squared_error(y_test, challenger_model.predict(X_test))

    print(f"Champion RMSE: {champion_rmse}")
    print(f"Challenger RMSE: {challenger_rmse}")

    # Decidir si promover el challenger a champion
    if challenger_rmse < champion_rmse * 0.95:
        print("El challenger supera significativamente al champion y debería ser promovido.")
        client.set_registered_model_alias("nyc-taxi-model", "champion", model_version.version)
    else:
        print("El challenger no supera significativamente al champion. Mantenemos el champion actual.")
else:
    print("No se pudieron cargar uno o ambos modelos. Verifica que existan en el Model Registry.")

# Si no hay un modelo champion, podríamos querer promover el challenger automáticamente
if not champion_model and challenger_model:
    print("No existe un modelo champion. Promoviendo el challenger a champion.")
    client.set_registered_model_alias("nyc-taxi-model", "champion", model_version.version)

# Asegúrate de que el challenger siempre se registre
if challenger_model:
    client.set_registered_model_alias("nyc-taxi-model", "challenger", model_version.version)
else:
    print("No se pudo registrar el modelo challenger. Verifica el proceso de entrenamiento y registro.")

In [ ]:
# Decidir si promover el challenger a champion
if challenger_rmse < champion_rmse * 0.95:
    print("El challenger supera significativamente al champion y debería ser promovido.")
    client.set_registered_model_alias("nyc-taxi-model", "champion", model_version.version)
else:
    print("El challenger no supera significativamente al champion. Mantenemos el champion actual.")


In [ ]:
# Subir datos a DagsHub
s3 = get_repo_bucket_client("colome8/nyc-taxi-time-prediction")

s3.upload_file(
    Bucket="nys-taxi-time-prediction",
    Filename="../data/green_tripdata_2024-03.parquet",
    Key="test_data.parquet",
)